In [ ]:
# Импорт библиотек
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
import string

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression

In [ ]:
#скачиваем датасет
train = pd.read_csv('../input/nlp-getting-started/train.csv')
test = pd.read_csv('../input/nlp-getting-started/test.csv')
train.head()

In [ ]:
sentences = ['The weather is sunny', 'The weather is partly sunny and partly cloudy.']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(sentences)
vectorizer.vocabulary_
vectorizer.transform(sentences).toarray()

In [ ]:
#стопслова
stopwords = stopwords.words('english')

count_vectorizer = CountVectorizer(stop_words = stopwords)
count_vectorizer.fit(train['text'])

train_vectors = count_vectorizer.transform(train['text'])
test_vectors = count_vectorizer.transform(test['text'])

train_vectors.shape

In [ ]:
#mindf/maxdf
count_vectorizer = CountVectorizer(stop_words = stopwords, min_df=2 ,max_df=0.8)
count_vectorizer.fit(train['text'])

train_vectors = count_vectorizer.transform(train['text'])
test_vectors = count_vectorizer.transform(test['text'])

In [ ]:
# Убираем лишние символы регулярками

def custom_preprocessor(text):

    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

    

In [ ]:
count_vectorizer = CountVectorizer(list(train['text']),preprocessor=custom_preprocessor)

train_vectors = count_vectorizer.fit_transform(list(train['text']))
test_vectors = count_vectorizer.transform(list(test['text']))
 

In [ ]:
count_vectorizer = CountVectorizer(list(train['text']),preprocessor=custom_preprocessor,ngram_range=(1,2))

train_vectors = count_vectorizer.fit_transform(list(train['text']))
test_vectors = count_vectorizer.transform(list(test['text']))

list(count_vectorizer.vocabulary_)[:10]

In [ ]:
count_vectorizer = CountVectorizer(list(train['text']),preprocessor=custom_preprocessor,ngram_range=(1,3),
                                  analyzer='char_wb')

train_vectors = count_vectorizer.fit_transform(list(train['text']))
test_vectors = count_vectorizer.transform(list(test['text']))

print(list(count_vectorizer.vocabulary_)[:20])

In [ ]:
count_vectorizer = CountVectorizer(token_pattern=r'\w{1,}',
                   ngram_range=(1, 2), stop_words = stopwords,preprocessor=custom_preprocessor)
count_vectorizer .fit(train['text'])

train_vectors = count_vectorizer.transform(train['text'])
test_vectors = count_vectorizer.transform(test['text'])

In [ ]:
#logistic Regression sklearn
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(clf, train_vectors, train["target"], cv=5, scoring="f1")
scores

In [ ]:
clf.fit(train_vectors, train["target"])

In [ ]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission["target"] = clf.predict(test_vectors)
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
#TF-IDF
tfidf = TfidfVectorizer(analyzer='word',token_pattern=r'\w{1,}',max_features=5000)
train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test["text"])

In [ ]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(2,3),token_pattern=r'\w{1,}',max_features=5000)
train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test["text"])

In [ ]:
tfidf = TfidfVectorizer(analyzer='char',ngram_range=(1,3),token_pattern=r'\w{1,}',max_features=5000)
train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test["text"])


In [ ]:
#model
tfidf_vectorizer = TfidfVectorizer( min_df=3,  max_features=None,analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = stopwords)

train_tfidf = tfidf.fit_transform(train['text'])
test_tfidf = tfidf.transform(test["text"])


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=1.0)
scores = model_selection.cross_val_score(clf, train_tfidf, train["target"], cv=5, scoring="f1")
scores

In [ ]:
# Logistic Regression + TF-IDF
clf.fit(train_tfidf, train["target"])